<a href="https://colab.research.google.com/github/gouthamkallempudi/googletechdevguide/blob/master/Lesson10_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import keras
keras.__version__

'2.2.4'

In [0]:
from keras.datasets import boston_housing
(train_data , train_targets) , (test_data , test_targets) = boston_housing.load_data()


In [13]:
train_data.shape

(404, 13)

In [14]:
test_data.shape

(102, 13)

In [0]:
# The 13 features in the input data are as follow:

# Per capita crime rate.
# Proportion of residential land zoned for lots over 25,000 square feet.
# Proportion of non-retail business acres per town.
# Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
# Nitric oxides concentration (parts per 10 million).
# Average number of rooms per dwelling.
# Proportion of owner-occupied units built prior to 1940.
# Weighted distances to five Boston employment centres.
# Index of accessibility to radial highways.
# Full-value property-tax rate per $10,000.
# Pupil-teacher ratio by town.
# 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
# % lower status of the population.

In [0]:
#Normalization

mean = train_data.mean(axis = 0)
train_data = train_data - mean

std = train_data.std(axis = 0)
train_data = train_data/std


test_data -= mean
test_data /= std

In [0]:
from keras.models import Sequential
from keras.layers import Dense

def build_model():
  model = Sequential()
  model.add(Dense(64 , activation = "relu" , input_shape = (train_data.shape[1] , )))
  model.add(Dense(64 , activation = "relu"))
  model.add(Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [18]:
#K-fold validation
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
  print("processed fold #" , i)
  
  #validation data
  val_data = train_data[ i * num_val_samples : (i+1) * num_val_samples ]
  val_targets = train_targets[ i * num_val_samples : (i+1) * num_val_samples ]
  
  #training data
  partial_train_data = np.concatenate([train_data[ : i * num_val_samples ] , train_data[(i+1)*num_val_samples : ]],axis = 0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i+1) * num_val_samples:]],axis=0)
  
  model = build_model()
  model.fit(partial_train_data , partial_train_targets , epochs= num_epochs , batch_size = 1 , verbose=0 )
  val_mse, val_mae =  model.evaluate(val_data , val_targets , verbose = 0)
  all_scores.append(val_mae)

processed fold # 0
processed fold # 1
processed fold # 2
processed fold # 3


In [19]:
print(all_scores)

[2.0638657194553036, 2.278493597955987, 2.7852662010948257, 2.3534523992255183]


In [20]:
np.mean(all_scores)

2.370269479432909

In [0]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [22]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [0]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]